In [1]:
%cd ..

/Users/Matteo/PycharmProjects/TrafficEmu


# Bayesian optimization: finding the minimum time loss

In this notebook we will performing bayesian optimization on the emulator trained on our sumo simulation. We are interested in identifying the parameters which enable us to obtain the minimum travel time. Our emulator is a gaussian process with an **rbf** kernel.


## Imports

In [2]:
import pickle
import numpy as np
import emukit as ek
import GPy
import pandas as pd

from emukit.model_wrappers import GPyModelWrapper
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from experimental_design import config
from sumo_grid_simulation.grid_simulation import Simulator

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


## Simulator

We start by initialising our simulation.

In [3]:
simulator = Simulator(end_time=300)

We then define the two user functions in which we are interested:

In [4]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average time loss / average route duration
        result.append(s['timeLoss']/s['duration'])
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

We then load the parameters' space from our configuration file:

In [5]:
parameter_space = config.get_parameter_space()

## Emulator

### Rnadomly initialised emulator

The first emulator we are going to analyse is the one that is just initilised with random points

We first sample at random 200 datapoints from the parameter space:

In [6]:
init_X, init_Y = pickle.load(open('bayesian_optimization/init_points/270_init_points_timeLoss.pkl', 'rb'))

'''
design = RandomDesign(parameter_space)
num_init_points = 270
init_X = design.get_samples(num_init_points)
init_Y = user_function_time_loss(init_X)
init_points = init_X, init_Y
with open(f'bayesian_optimization/init_points/{num_init_points}_init_points_timeLoss.pkl', "wb") as f:
     pickle.dump(init_points, f)
'''

'\ndesign = RandomDesign(parameter_space)\nnum_init_points = 270\ninit_X = design.get_samples(num_init_points)\ninit_Y = user_function_time_loss(init_X)\ninit_points = init_X, init_Y\nwith open(f\'bayesian_optimization/init_points/{num_init_points}_init_points_timeLoss.pkl\', "wb") as f:\n     pickle.dump(init_points, f)\n'

To then fit a GP to these points

In [7]:
emulator = GPy.models.GPRegression(init_X, init_Y, noise_var=1e-10)
emukit_model_random_init = GPyModelWrapper(emulator, n_restarts=5)
emukit_model_random_init.optimize()

Optimization restart 1/5, f = -430.9457829821215
Optimization restart 2/5, f = -513.9188950292898
Optimization restart 3/5, f = -513.9188950240906
Optimization restart 4/5, f = 58.987090567108226
Optimization restart 5/5, f = -513.9188950288074


### Experimentally designed emulator with model variance

We also load the emulator obtained during experimental design:

In [8]:
emukit_model_variance = pickle.load(open('experimental_design/models/model_variance_20_init_points_250_loops_timeloss.pkl', 'rb'))

### Experimentally designed emulator with integrated variance reduction

We also load the emulator obtained during experimental design:

In [9]:
emukit_model_integrated_variance = pickle.load(open('experimental_design/models/integrated_variance_reduction_20_init_points_250_loops_timeloss.pkl', 'rb'))

## Bayesian optimisation

We now run bayesian optimisation on the emulators

In [10]:
n_iter_bo = 50

### Acquisition functions

In [11]:
acquisition_random_init = ExpectedImprovement(emukit_model_random_init)

In [12]:
acquisition_model_variance = ExpectedImprovement(emukit_model_variance)

In [13]:
acquisition_integrated_variance = ExpectedImprovement(emukit_model_integrated_variance)

### Optimisation loop

In [14]:
bo_random_init = BayesianOptimizationLoop(parameter_space, emukit_model_random_init, acquisition=acquisition_random_init)
bo_random_init.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -513.9188950294603
Optimization restart 2/5, f = -513.9188950279458
Optimization restart 3/5, f = -513.9188950295323
Optimization restart 4/5, f = -513.9188950295388
Optimization restart 5/5, f = -513.9188950292851
[[15.  8. 27. 70.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.18167094074773923

Optimization restart 1/5, f = -510.0560464869394
Optimization restart 2/5, f = -510.0560464789539
Optimization restart 3/5, f = -510.056046486293
Optimization restart 4/5, f = 58.88741225058149


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = -492.5045910496916
[[20.         20.91362833  5.         49.70555328  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08534389629096147

Optimization restart 1/5, f = -512.4415640695063
Optimization restart 2/5, f = -512.4415640655757
Optimization restart 3/5, f = -512.4415640701556
Optimization restart 4/5, f = -512.4415640694019
Optimization restart 5/5, f = -512.4415640708618
[[20.  25.   5.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08285966906656562

Optimization restart 1/5, f = -514.3266544124491
Optimization restart 2/5, f = -514.3266544124384
Optimization restart 3/5, f = -514.3266544116537
Optimization restart 4/5, f = 58.39711579277022
Optimization restart 5/5, f = 58.39712103135071
[[20.         8.         5.        31.7081038  1.         1.5      ]]

User fun

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = -515.024972146018
Optimization restart 4/5, f = -535.0311602524763
Optimization restart 5/5, f = 56.067642883469716
[[14.   8.   5.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08420152802265321

Optimization restart 1/5, f = -537.5248302729931
Optimization restart 2/5, f = -537.5248478436549
Optimization restart 3/5, f = -537.5248478438607
Optimization restart 4/5, f = 55.805764685567624
Optimization restart 5/5, f = -537.5248478441184
[[17.  25.   5.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08405229920839632

Optimization restart 1/5, f = -540.0882501053561
Optimization restart 2/5, f = -540.0882584561557
Optimization restart 3/5, f = -540.0882584538233
Optimization restart 4/5, f = -540.0882584563136
Optimization restart 5/5, f = -540.0882525938474
[[20.          8.     

...

Optimization restart 5/5, f = -527.9945060789906
[[20.          8.          5.         54.64335333  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.1066714879768577

Optimization restart 1/5, f = -544.7343138547094
Optimization restart 2/5, f = 55.041063012086056
Optimization restart 3/5, f = -544.7343137436465
Optimization restart 4/5, f = 55.04105613057297
Optimization restart 5/5, f = 55.04106312473141
[[ 3.  25.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.09140625

Optimization restart 1/5, f = -546.0886491860399
Optimization restart 2/5, f = -546.0886491913589
Optimization restart 3/5, f = -546.0886491910287
Optimization restart 4/5, f = -546.0886491566089
Optimization restart 5/5, f = -546.0886491541407
[[20.         25.          5.         55.02603343  1.          1.5       ]]

User functio

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 4/5, f = -551.4535271849365
Optimization restart 5/5, f = -551.4535271837944
[[17.   8.   5.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08405229920839632

Optimization restart 1/5, f = -554.2101666030057
Optimization restart 2/5, f = -554.2101666067047
Optimization restart 3/5, f = -554.2101660844585
Optimization restart 4/5, f = -554.2101666067331
Optimization restart 5/5, f = -554.2101666067474
[[14.         20.03942125  5.         39.20388973  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.09092632976201763

Optimization restart 1/5, f = -556.9794093991749
Optimization restart 2/5, f = -556.9794093972369
Optimization restart 3/5, f = -556.9794094050817
Optimization restart 4/5, f = -556.9794094040287
Optimization restart 5/5, f = -556.9794094051101
[[17.  25.   5.  70.   1.   

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 4/5, f = -583.8886472491549
Optimization restart 5/5, f = -583.8886472491447
[[20.  25.   5.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08285966906656562

Optimization restart 1/5, f = -586.6629136899433
Optimization restart 2/5, f = -586.6629136867284
Optimization restart 3/5, f = -586.6629136892434
Optimization restart 4/5, f = -519.3711956236871
Optimization restart 5/5, f = -586.6629121732175
[[10.   8.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.09022680412371134

Optimization restart 1/5, f = -589.3423559195891
Optimization restart 2/5, f = -522.0697353390418
Optimization restart 3/5, f = -589.3423572853734
Optimization restart 4/5, f = -522.0697353814533
Optimization restart 5/5, f = -522.0697353813658
[[18.         25.          5.27374157 70.          1.          1.5    

In [15]:
bo_model_variance = BayesianOptimizationLoop(parameter_space, emukit_model_variance, acquisition=acquisition_model_variance)
bo_model_variance.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -414.61439874911133
Optimization restart 2/5, f = -355.8677674583225
Optimization restart 3/5, f = -355.86777669859805
Optimization restart 4/5, f = -355.8677766883393
Optimization restart 5/5, f = -414.6143987463941
[[20.         15.93051444  5.         41.5815497   1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.0888494318181818

Optimization restart 1/5, f = -416.7636091937443
Optimization restart 2/5, f = -356.8677999613101
Optimization restart 3/5, f = -356.86779946671084
Optimization restart 4/5, f = -356.86779367977795
Optimization restart 5/5, f = -356.86779996101683
[[14.         25.          5.         63.21670736  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.0879094119079508

Optimization restart 1/5, f = -419.07074598361487
Optimization restart 2/5, f = -

 Retrying in 1 seconds

Output 0.08507618056212737

Optimization restart 1/5, f = -459.7783568762581
Optimization restart 2/5, f = -321.50793079095206
Optimization restart 3/5, f = -400.8046344168415
Optimization restart 4/5, f = -400.80463479633363
Optimization restart 5/5, f = -400.8046347814227
[[15.         21.12915788  5.         62.10917896  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08520405307059285

Optimization restart 1/5, f = -462.55482368592186
Optimization restart 2/5, f = -462.5548236806799
Optimization restart 3/5, f = -462.55482368136023
Optimization restart 4/5, f = -245.3055222675899
Optimization restart 5/5, f = -405.9896918712329
[[20.          8.          5.         53.31571183  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.10775320432269414

Optimization restart 1/5, f 

 Retrying in 1 seconds

Output 0.09235545207605743

Optimization restart 1/5, f = -509.67994267395363
Optimization restart 2/5, f = -471.64138348375695
Optimization restart 3/5, f = -471.641383487475
Optimization restart 4/5, f = -471.6413834894526
Optimization restart 5/5, f = -471.64138348187566
[[13.  25.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08743373869660119

Optimization restart 1/5, f = -512.4351334631621
Optimization restart 2/5, f = -512.4351334383421
Optimization restart 3/5, f = -512.4351334630712
Optimization restart 4/5, f = -474.94446458127504
Optimization restart 5/5, f = -474.9444645632055
[[20.         21.95559972  5.         58.9651855   3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.0877578626986811

Optimization restart 1/5, f = -515.2890330925879
Optimization restart 2/

In [16]:
bo_integrated_variance = BayesianOptimizationLoop(parameter_space, emukit_model_integrated_variance, acquisition=acquisition_integrated_variance)
bo_integrated_variance.run_loop(user_function_time_loss, n_iter_bo)

Optimization restart 1/5, f = -452.57416906316786
Optimization restart 2/5, f = -452.5741690631713
Optimization restart 3/5, f = -452.57416906316973
Optimization restart 4/5, f = -452.57416906315666
Optimization restart 5/5, f = -452.574169058188
[[13.  8. 27. 70.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.17959999999999998

Optimization restart 1/5, f = -451.5219356734176
Optimization restart 2/5, f = -451.52193569164535
Optimization restart 3/5, f = -451.5219356916381
Optimization restart 4/5, f = -451.52193569162574
Optimization restart 5/5, f = -451.5219356914573
[[13.          8.          5.         44.29860688  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.09103727857583502

Optimization restart 1/5, f = -453.75083819342336
Optimization restart 2/5, f = -453.7508381914124
Optimization restart 3/5, f =

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 5/5, f = -475.43157541829385
[[20.         16.67436597  5.         70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08285966906656562

Optimization restart 1/5, f = -482.5435090793595
Optimization restart 2/5, f = -482.54350906880336
Optimization restart 3/5, f = -482.5435090799602
Optimization restart 4/5, f = -482.54350907919047
Optimization restart 5/5, f = -415.80389970873443
[[ 3.  25.   5.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.09576642335766422

Optimization restart 1/5, f = -484.8053196470339
Optimization restart 2/5, f = -484.80535906828425
Optimization restart 3/5, f = -484.80535907204285
Optimization restart 4/5, f = -417.85103171052253
Optimization restart 5/5, f = -484.8053590693868
[[20.         16.97121144  5.         70.          1.          1.5    

 Retrying in 1 seconds

Output 0.08285966906656562

Optimization restart 1/5, f = -526.7918951629401
Optimization restart 2/5, f = -514.1117310691412
Optimization restart 3/5, f = -526.7918951640249
Optimization restart 4/5, f = -514.1117315697887
Optimization restart 5/5, f = -514.1117304866011
[[20.          8.          5.         66.74562939  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08342877279534842

Optimization restart 1/5, f = -529.3307336979966
Optimization restart 2/5, f = -520.2265732412917
Optimization restart 3/5, f = -520.226585546822
Optimization restart 4/5, f = -520.2265852166615
Optimization restart 5/5, f = -520.2265863207493
[[20.   8.   5.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08285966906656562

Optimization restart 1/5, f = -531.8650811264117
Optimization restart 2/5, 

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = -543.1399821743435
[[ 3.   8.   5.  70.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.09576642335766422

Optimization restart 1/5, f = -544.5662867974751
Optimization restart 2/5, f = -544.5662773285591
Optimization restart 3/5, f = -544.566286798588
Optimization restart 4/5, f = -544.566257039557
Optimization restart 5/5, f = -548.6402146709661
[[20.         18.96238261  5.         70.          1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 0.08285966906656562

Optimization restart 1/5, f = -551.2418926361305
Optimization restart 2/5, f = -552.6501888422878
Optimization restart 3/5, f = -551.2418926364624
Optimization restart 4/5, f = -552.6501889031132
Optimization restart 5/5, f = -551.2418926363516
[[ 3.         15.02241021  5.         65.98471087  1.          1.5       ]]

Us

## Results

In [17]:
parameters_name = ['Size of the grid', 'Speed limit in the net (m/s)', 'Max vehicles speed (m/s)', 'Length of the roads (m)', 'Number of lanes', 'Accelleration (m/s^2)']

### Results on the random initialised emulator

In [18]:
results_random_init = bo_random_init.get_results()

In [19]:
results_random_init_df = pd.DataFrame(results_random_init.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,18.0,25.0,5.292831,62.432391,1.0,1.5


In [20]:
results_random_init.minimum_value*100

8.00330433937509

### Results on the experimentally designed emulator with model variance

In [21]:
results_model_variance = bo_model_variance.get_results()

In [22]:
results_random_init_df = pd.DataFrame(results_model_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,13.0,8.0,6.77514,70.0,1.0,1.5


In [23]:
results_model_variance.minimum_value*100

8.105431553287119

### Results on the experimentally designed emulator with integrated variance reduction

In [24]:
results_integrated_variance = bo_integrated_variance.get_results()

In [25]:
results_random_init_df = pd.DataFrame(results_integrated_variance.minimum_location.reshape(1,-1))
results_random_init_df.columns = parameters_name
results_random_init_df

,Size of the grid,Speed limit in the net (m/s),Max vehicles speed (m/s),Length of the roads (m),Number of lanes,Accelleration (m/s^2)
0,20.0,8.0,5.0,70.0,1.0,1.5


In [26]:
results_integrated_variance.minimum_value*100

8.285966906656563

## Save stuff

In [27]:
with open(f'bayesian_optimization/bayesian_opt_results/results_random_init_timeLoss.pkl', "wb") as f:
     pickle.dump(results_random_init, f)

with open(f'bayesian_optimization/bayesian_opt_results/results_model_variance_timeLoss.pkl', "wb") as f:
     pickle.dump(results_model_variance, f)
        
with open(f'bayesian_optimization/bayesian_opt_results/results_integrated_variance_timeLoss.pkl', "wb") as f:
     pickle.dump(results_integrated_variance, f)

In [28]:
with open(f'bayesian_optimization/models/270_random_init_50_bay_opt_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_random_init, f)

with open(f'bayesian_optimization/models/270_model_variance_50_bay_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_variance, f)
        
with open(f'bayesian_optimization/models/270_integrated_variance_50_bay_timeLoss.pkl', "wb") as f:
     pickle.dump(emukit_model_integrated_variance, f)